In [1]:
import time
from sklearn.pipeline import make_pipeline
time_start = time.time()
import re
import os
import numpy as np
import scipy as sp
import mne
import pickle as pkl
import nilearn.plotting as nplt
import nilearn.image as image
import pickle
from mne.preprocessing import (ICA, create_eog_epochs, create_ecg_epochs, corrmap)
from sklearn.metrics import roc_auc_score
from nodestimation.node_estimate import Node
from nodestimation.timewindow import TimeWindow, sliding_window
import nibabel as nib
import matplotlib.pyplot as plt
import nodestimation.project.path as path
from nodestimation.project.subject import Subject
from nodestimation.project.path import  find_subject_dir
from nodestimation.pipeline import pipeline
from sklearn.model_selection import train_test_split
import sklearn.metrics as sm
import pandas as pd
import matplotlib.pyplot as plt
from nodestimation.project.actions import read
from nodestimation.mlearning.features import prepare_data
import nodestimation as nd
from nodestimation.project.structures import subject_data_types

subjects = pipeline(methods=['coh', 'imcoh', 'plv', 'ciplv', 'ppc', 'pli', 'psd', 'envelope', 'pearson'],
                    freq_bands=[(0.5, 4), (4, 7), (7, 14), (14, 30)])

Building of the resources files tree...
Analysing project structure...
	Subject:  B1C2
		Trans file: ok
		Fwd file: ok
		Src file: ok
		Bem file: ok
		Raw file: ok
		Ave file: ok
		Stc file: ok
		Epo file: ok
		Parc file: ok
		Coords file: ok
		Resec_mni file: ok
		Inv file: ok
		Cov file: ok
		Eve file: ok
		Nodes file: ok
		Feat file: ok
		Resec file: ok
	Files structure for B1C2 has been analysed. Files tree is built
Preparing data...
Looking for original raw file in files tree...
Original raw file has been found; trying to read...
3 files found: ['./Source/Subjects/B1C2/B1C2_ii_run1_raw_tsss_mc_art_corr_raw.fif', './Source/Subjects/B1C2/a834165552547eb79a9c1f0cab4efdb3/B1C2_node_estimation_pipeline_file_first_processing_output_raw.fif', './Source/Subjects/B1C2/d1200838a4634e09bbce0a67ee4d9ed3/B1C2_node_estimation_pipeline_file_first_processing_output_raw.fif']. Only the original files are going to be read
Opening raw data file ./Source/Subjects/B1C2/B1C2_ii_run1_raw_tsss_mc_art_cor

/home/ubuntu/PycharmProjects/NodesEstimation/nodestimation/project/path.py:160: RuntimeWarning: This filename (./Source/Subjects/B1C2/source_spaces_ico5.fif) does not conform to MNE naming conventions. All source space files should end with -src.fif, -src.fif.gz, _src.fif, _src.fif.gz, -fwd.fif, -fwd.fif.gz, _fwd.fif, _fwd.fif.gz, -inv.fif, -inv.fif.gz, _inv.fif or _inv.fif.gz
  read[type](tree[type][i])
/home/ubuntu/PycharmProjects/NodesEstimation/nodestimation/project/path.py:160: RuntimeWarning: This filename (./Source/Subjects/B1C2/source_spaces_oct5.fif) does not conform to MNE naming conventions. All source space files should end with -src.fif, -src.fif.gz, _src.fif, _src.fif.gz, -fwd.fif, -fwd.fif.gz, _fwd.fif, _fwd.fif.gz, -inv.fif, -inv.fif.gz, _inv.fif or _inv.fif.gz
  read[type](tree[type][i])
/home/ubuntu/PycharmProjects/NodesEstimation/nodestimation/project/path.py:160: RuntimeWarning: This filename (./Source/Subjects/B1C2/forward_ico5.fif) does not conform to MNE naming c

In [2]:
feat = read['feat']('Source/Subjects/B1C2/d1200838a4634e09bbce0a67ee4d9ed3/B1C2_node_estimation_pipeline_file_features_computation_output_feat.pkl')

print(feat['time-domain']['pearson'].shape)

(68, 68)


In [6]:
print(subjects[0].nodes[0].features.keys())
print(subjects[0].nodes[0].features['0.5-4Hz'].keys())
print(subjects[0].nodes[0].features['time-domain'].keys())

dict_keys(['0.5-4Hz', '4-7Hz', '7-14Hz', '14-30Hz', 'time-domain'])
dict_keys(['coh', 'imcoh', 'plv', 'ciplv', 'ppc', 'pli', 'psd'])
dict_keys(['pearson', 'envelope'])


In [7]:
print(subjects[0].nodes[0].features)
print(subjects[0].nodes[1].features)
print(subjects[0].nodes[2].features)

{'0.5-4Hz': {'coh': -1.317363485577068, 'imcoh': -1.3394734727204665, 'plv': -2.0528235875580663, 'ciplv': -1.5525562836199378, 'ppc': -1.678543701052078, 'pli': -1.6399054623725264, 'psd': 36.43435844120942}, '4-7Hz': {'coh': -1.3642047636778902, 'imcoh': -1.0023628028600178, 'plv': -2.284270829462227, 'ciplv': -0.8920533966697122, 'ppc': -1.8255463624718395, 'pli': -1.1359849016943082, 'psd': 29.832568642664928}, '7-14Hz': {'coh': -1.3239278298736006, 'imcoh': -0.8657717860812277, 'plv': -2.3112786450743776, 'ciplv': -0.7436323127248022, 'ppc': -1.8821620195385906, 'pli': -0.9687787276170354, 'psd': 50.98730836305875}, '14-30Hz': {'coh': -1.2904217457516134, 'imcoh': -0.8168531814726282, 'plv': -2.283747717428443, 'ciplv': -0.6243336761910843, 'ppc': -1.941641352449933, 'pli': -0.8212089575771051, 'psd': 30.677948564482378}, 'time-domain': {'pearson': -0.9878713299550234, 'envelope': -0.477310852682067}}
{'0.5-4Hz': {'coh': -1.1454248655967996, 'imcoh': -0.9750095919240707, 'plv': -1

In [2]:
print(subjects[0].nodes[0].features)
print(subjects[0].nodes[1].features)
print(subjects[0].nodes[2].features)

{'0.5-4Hz': {'coh': -1.317363485577068, 'imcoh': -1.3394734727204665, 'plv': -2.0528235875580663, 'ciplv': -1.5525562836199378, 'ppc': -1.678543701052078, 'pli': -1.6399054623725264, 'psd': 36.43435844120942, 'envelope': -0.477310852682067}, '4-7Hz': {'coh': -1.3642047636778902, 'imcoh': -1.0023628028600178, 'plv': -2.284270829462227, 'ciplv': -0.8920533966697122, 'ppc': -1.8255463624718395, 'pli': -1.1359849016943082, 'psd': 29.832568642664928, 'envelope': -0.477310852682067}, '7-14Hz': {'coh': -1.3239278298736006, 'imcoh': -0.8657717860812277, 'plv': -2.3112786450743776, 'ciplv': -0.7436323127248022, 'ppc': -1.8821620195385906, 'pli': -0.9687787276170354, 'psd': 50.98730836305875, 'envelope': -0.477310852682067}, '14-30Hz': {'coh': -1.2904217457516134, 'imcoh': -0.8168531814726282, 'plv': -2.283747717428443, 'ciplv': -0.6243336761910843, 'ppc': -1.941641352449933, 'pli': -0.8212089575771051, 'psd': 30.677948564482378, 'envelope': -0.477310852682067}}
{'0.5-4Hz': {'coh': -1.1454248655

In [ ]:
subjects = pipeline(con_method=['coh', 'imcoh', 'plv', 'ciplv', 'ppc', 'pli'],
                    freq_diaps=[(0.5, 4), (4, 7), (7, 14), (14, 30), (30, 70)])
psd = subjects[0].nodes[0].features['4-7Hz']['psd']
plt.plot(psd)
plt.show()
psd = nd.mlearning.features.iterp_for_psd(psd, 68)
plt.plot(psd)
plt.show()

In [ ]:
# x_train, x_test, y_train, y_test = train_test_split(a['B1C2']['X'], a['B1C2']['Y'])
# from sklearn.svm import SVR
# from sklearn.pipeline import make_pipeline
# from sklearn.preprocessing import StandardScaler
#
# regr = make_pipeline(StandardScaler(), SVR(C=1.0, epsilon=0.2))
# regr.fit(x_train, y_train)
# y_pred = regr.predict(x_test)
#
# print(y_pred)
# print()
# print(y_test)

In [ ]:
from nodestimation.mlearning.features import prepare_connectivity, prepare_data
con = pkl.load(open(subjects[0].data['con'], 'rb'))

In [ ]:
# fig, ax = plt.subplots()
# ax.plot(psd['0.5-4Hz']['bankssts-lh']['freq'], psd['0.5-4Hz']['bankssts-lh']['psd'].mean(axis=0))
# # ax.set_xlabel('Freq (Hz)')
# ax.set_xlim(psd['0.5-4Hz']['bankssts-lh']['freq'][[0, -1]])
# ax.set_ylabel('Power Spectral Density')

In [ ]:
subjects_dir, subjects_ = find_subject_dir()
labels = mne.read_labels_from_annot('B1C2', parc='aparc', subjects_dir=subjects_dir)
label_names = [label.name for label in labels]
lh_labels = [name for name in label_names if name.endswith('lh')]
rh_labels = [name for name in label_names if name.endswith('rh')]

label_ypos_lh = list()

for name in lh_labels:
    idx = label_names.index(name)
    ypos = np.mean(labels[idx].pos[:, 1])
    label_ypos_lh.append(ypos)

try:
    idx = label_names.index('Brain-Stem')

except ValueError:
    pass

else:
    ypos = np.mean(labels[idx].pos[:, 1])
    lh_labels.append('Brain-Stem')
    label_ypos_lh.append(ypos)


lh_labels = [label for (yp, label) in sorted(zip(label_ypos_lh, lh_labels))]

rh_labels = [label[:-2] + 'rh' for label in lh_labels
             if label != 'Brain-Stem' and label[:-2] + 'rh' in rh_labels]


node_colors = [label.color for label in labels]

node_order = lh_labels[::-1] + rh_labels

node_angles = mne.viz.circular_layout(label_names, node_order, start_pos=90,
                              group_boundaries=[0, len(label_names) // 2])

In [ ]:
methods = [
    'coh',
    'imcoh',
    'plv',
    'ciplv',
    'ppc',
    'pli'
]

freq_bands = [
    '0.5-4Hz',
    '4-7Hz',
    '7-14Hz',
    '14-30Hz'
]

for method in methods:
    for freq_band in freq_bands:
        fig = plt.figure(num=None, figsize=(25, 25), facecolor='black')
        mne.viz.plot_connectivity_circle(con[freq_band][method]['con'][:, :, 0], label_names, n_lines=300,
                                         node_angles=node_angles, node_colors=node_colors,
                                         title='All-to-All Connectivity Epilepsy Condition ({} for {})'
                                         .format(method, freq_band), padding=8, fontsize_title=35, fontsize_colorbar=25,
                                         fontsize_names=20, fig=fig
                                         )

In [ ]:
raw = read['raw'](subjects[0].data['raw_fp'])
eve = read['eve'](subjects[0].data['eve'])
epo = read['epo'](subjects[0].data['epo'])
ave = read['ave'](subjects[0].data['ave'])
inv = read['inv'](subjects[0].data['inv'])

raw.plot(events=eve, color='gray',
         event_color={1: 'r'})

In [ ]:
ave[0].plot()

In [ ]:
epo.plot()

In [ ]:
epo.plot_psd()

In [ ]:
epo.plot_image()

In [ ]:
stc = read['stc'](subjects[0].data['stc'])

In [ ]:
src = read['src'](subjects[0].data['src'][0])

In [ ]:
label_ts = mne.extract_label_time_course(stc, labels, src, mode='mean_flip')

In [ ]:
plt.plot(label_ts[0][0])
plt.show()
plt.plot(label_ts[1][0])
plt.show()
plt.plot(label_ts[2][0])
plt.show()
plt.plot(label_ts[3][0])
plt.show()

In [ ]:
for c in con['0.5-4Hz']['plv']['con'][:,:,0]:
    print(c[0:5])
print('2:3 ', con['0.5-4Hz']['plv']['con'][3,2,0])

plt.imshow(con['0.5-4Hz']['plv']['con'][:,:,0] + con['0.5-4Hz']['plv']['con'][:,:,0].T)
plt.show()
plt.plot(np.arange(401), label_ts[0][2].T, np.arange(401), label_ts[0][3].T)
plt.show()
plt.plot(np.arange(401), label_ts[1][2].T, np.arange(401), label_ts[1][3].T)
plt.show()
plt.plot(np.arange(401), label_ts[2][2].T, np.arange(401), label_ts[2][3].T)
plt.show()
plt.plot(np.arange(401), label_ts[3][2].T, np.arange(401), label_ts[3][3].T)
plt.show()

In [ ]:
print('0:11 ', con['0.5-4Hz']['plv']['con'][11,0,0])

plt.plot(np.arange(401), label_ts[0][11].T, np.arange(401), label_ts[0][0].T)
plt.show()
plt.plot(np.arange(401), label_ts[1][11].T, np.arange(401), label_ts[1][0].T)
plt.show()
plt.plot(np.arange(401), label_ts[2][11].T, np.arange(401), label_ts[2][0].T)
plt.show()
plt.plot(np.arange(401), label_ts[3][11].T, np.arange(401), label_ts[3][0].T)
plt.show()